In [1]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv("Data.csv", sep=";")
df.dtypes

Direksi Perempuan    int64
Direksi Laki-Laki    int64
Harga penutupan      int64
Total Shares         int64
Total Utang          int64
Total Asset          int64
dtype: object

In [3]:
#X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
Y = np_plus_liability / df["Total Asset"]
#Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [4]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,150.000000,150.000000
mean,1.326667,1.297296
std,1.217857,2.019696
min,0.000000,0.159993
25%,0.000000,0.930708
50%,1.000000,0.978282
75%,2.000000,1.141656
max,6.000000,18.327231


In [5]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [6]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.4319
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.512
Time:                        20:03:41   Log-Likelihood:                -317.56
No. Observations:                 150   AIC:                             639.1
Df Residuals:                     148   BIC:                             645.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4160      0.245      5.785      0.000       0.932       1.900
X             -0.0895      0.136     -0.657      0.512      -0.358       0.180
==============================================================================
Omnibus:                      257.108   Durbin-Watson:                   0.889
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27347.096
Skew:                           7.975   Prob(JB):                         0.00
Kurtosis:                      67.196   Cond. No.                         3.17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [8]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     157.4
Date:                Thu, 11 May 2023   Prob (F-statistic):           4.74e-25
Time:                        20:03:41   Log-Likelihood:                -281.48
No. Observations:                 150   AIC:                             567.0
Df Residuals:                     148   BIC:                             573.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7116      0.192      3.698      0.000       0.331       1.092
X              1.3427      0.107     12.546      0.000       1.131       1.554
==============================================================================
Omnibus:                      258.916   Durbin-Watson:                   1.125
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28388.447
Skew:                           8.069   Prob(JB):                         0.00
Kurtosis:                      68.435   Cond. No.                         3.17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [10]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [11]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

70    6
71    6
72    6
Name: X, dtype: int64

In [12]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [13]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [14]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [15]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [16]:
data_sample.describe()

,X,Y
count,124.000000,124.000000
mean,1.370968,1.011561
std,1.023844,0.125741
min,0.000000,0.688744
25%,1.000000,0.934392
50%,1.000000,0.972770
75%,2.000000,1.063231
max,4.000000,1.385464


In [17]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     2.672
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.105
Time:                        20:03:41   Log-Likelihood:                 83.015
No. Observations:                 124   AIC:                            -162.0
Df Residuals:                     122   BIC:                            -156.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9869      0.019     52.514      0.000       0.950       1.024
X              0.0180      0.011      1.635      0.105      -0.004       0.040
==============================================================================
Omnibus:                       16.763   Durbin-Watson:                   0.904
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.159
Skew:                           0.936   Prob(JB):                     6.91e-05
Kurtosis:                       3.449   Cond. No.                         3.56
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
bX = data_sample["X"] * result.params["X"]

In [19]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [20]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9361
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.335
Time:                        20:03:42   Log-Likelihood:                 145.48
No. Observations:                 124   AIC:                            -287.0
Df Residuals:                     122   BIC:                            -281.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0897      0.011      7.897      0.000       0.067       0.112
X              0.0064      0.007      0.968      0.335      -0.007       0.020
==============================================================================
Omnibus:                       39.839   Durbin-Watson:                   1.215
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               72.638
Skew:                           1.438   Prob(JB):                     1.69e-16
Kurtosis:                       5.405   Cond. No.                         3.56
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""